# Air Quality Analysis Ulm 2022

## Business Understanding

### Relevant Imports

In [1]:
import urllib
import pandas as pd
from pandas import json_normalize
from pandasql import sqldf
import json
import pprint
import sqlite3 as db

### Extract data from API

In [2]:
# get HTTP Response from url
url = 'https://datenhub.ulm.de/ckan/api/3/action/datastore_search?resource_id=b49de35e-040c-4530-9208-eefadc97b610&limit=206725'  
response = urllib.request.urlopen(url)
assert response.code == 200

In [3]:
# convert HTTP Response to json
string = response.read().decode('utf-8')
response_dict = json.loads(string)
assert response_dict['success'] is True

In [4]:
# convert JSON object to dataframe
created_package = response_dict['result']
df = pd.DataFrame.from_dict(created_package['records'])

In [5]:
# remove \r\n from string conversion
df = df.replace({r'\r\n': ''}, regex=True)

## Data Understanding

#### Beschreibung der Daten

station: Beschreibung des Messortes<br>
schadstoff: Schadstoffe in der Luft <br>
zeit_utc: Zeitstempel am Ende der Messung<br>
wert: Messwert in µg/m³ <br>

Schadstoffe
- NO2: Stickstoffdioxid
- O3: Ozon
- PM2.5: Feinstaub mit einem Durchmesser kleiner als 2,5µm
- PM10: Feinstaub mit einem Durchmesser kleiner als 10µm<br>


## Data Preperation

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206725 entries, 0 to 206724
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   _id         206725 non-null  int64  
 1   station     206725 non-null  object 
 2   schadstoff  206725 non-null  object 
 3   zeit_utc    206725 non-null  object 
 4   wert        206725 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 7.9+ MB


In [7]:
df['zeit_utc'] = df['zeit_utc'].replace({r'T': ' '}, regex=True)

In [8]:
df

,_id,station,schadstoff,zeit_utc,wert
0,1,Ulm Trollingerweg 2,NO2,2020-11-13 01:00:00,33.0
1,2,Ulm Trollingerweg 2,O3,2020-11-13 01:00:00,0.0
2,3,Ulm Trollingerweg 2,PM10,2020-11-13 01:00:00,26.0
3,4,Ulm in der Wanne,PM10,2020-11-13 01:00:00,22.0
4,5,Ulm Trollingerweg 2,PM2.5,2020-11-13 01:00:00,15.0
...,...,...,...,...,...
206720,206721,Ulm in der Wanne,PM2.5,2022-08-11 15:00:00,8.0
206721,206722,Ulm in der Wanne,NO2,2022-08-11 16:00:00,34.0
206722,206723,Ulm in der Wanne,O3,2022-08-11 16:00:00,99.0
206723,206724,Ulm in der Wanne,PM10,2022-08-11 16:00:00,24.0


In [9]:
df['zeit_utc'] = pd.to_datetime(df['zeit_utc'])

In [10]:
# separate date and time values
df['date'] = [d.date() for d in df['zeit_utc']]
df['time'] = [d.time() for d in df['zeit_utc']]

In [11]:
df.drop('zeit_utc', axis=1, inplace=True)

In [12]:
df

,_id,station,schadstoff,wert,date,time
0,1,Ulm Trollingerweg 2,NO2,33.0,2020-11-13,01:00:00
1,2,Ulm Trollingerweg 2,O3,0.0,2020-11-13,01:00:00
2,3,Ulm Trollingerweg 2,PM10,26.0,2020-11-13,01:00:00
3,4,Ulm in der Wanne,PM10,22.0,2020-11-13,01:00:00
4,5,Ulm Trollingerweg 2,PM2.5,15.0,2020-11-13,01:00:00
...,...,...,...,...,...,...
206720,206721,Ulm in der Wanne,PM2.5,8.0,2022-08-11,15:00:00
206721,206722,Ulm in der Wanne,NO2,34.0,2022-08-11,16:00:00
206722,206723,Ulm in der Wanne,O3,99.0,2022-08-11,16:00:00
206723,206724,Ulm in der Wanne,PM10,24.0,2022-08-11,16:00:00


## Modelling

In [16]:
query = "SELECT DISTINCT station FROM df"
sqldf(query, globals())

,station
0,Ulm Trollingerweg 2
1,Ulm in der Wanne
2,Ulm Maehringer Weg 95
3,Ulm Spielplatz Sonnenhalde
4,Ulm Spielplatz Eselsberg Fort


In [ ]:
query = "SELECT DISTINCT date FROM df"
sqldf(query, globals())

,date
0,2020-11-13
1,2020-11-14
2,2020-11-15
3,2020-11-16
4,2020-11-17
...,...
631,2022-08-07
632,2022-08-08
633,2022-08-09
634,2022-08-10


In [21]:
query = "SELECT schadstoff, MIN(wert) as min, MAX(wert) as max FROM df GROUP BY schadstoff"
sqldf(query, globals())

,schadstoff,min,max
0,NO2,5.0,81.0
1,O3,0.0,241.0
2,PM10,0.0,113.0
3,PM2.5,0.0,93.0


In [14]:
##### Evaluation